# Naive Bayes pour la classification du spam


La classification naïve bayésienne (Naive bayes) est un algorithme de classification probabiliste relativement simple qui convient bien aux données que l'on peut catégoriser.

En Machine Learning, les applications courantes de Naive Bayes sont la classification des courriels non sollicités (spam), l'analyse des sentiments (emotion analyses) et la catégorisation des documents (data categorisation). Naive Bayes présente des avantages par rapport aux autres algorithmes de classification couramment utilisés en raison de sa simplicité, de sa rapidité et de sa précision sur de petits ensembles de données.

## Data Description

Nous utiliserons les données du dépot d'apprentissage machine de l'UCI qui contient plusieurs commentaires Youtube de vidéos musicales très populaires. Chaque commentaire dans les données a été étiqueté comme spam ou ham (commentaire légitime), et nous utiliserons ces données pour former notre algorithme Naive Bayes pour la classification de spam commentaire youtube.

In [2]:
# Import modules

# Pour la manipulation des données
import pandas as pd

# Pour les opérations matricielles
import numpy as np

# Pour utiliser le regex
import re

In [3]:
# Charger les données du fichier 'YoutubeCommentsSpam.csv' en utilisant pandas
data_comments = 

# Créer des libellés de colonne : "content" et "label". 
# conseils : la méthode 'colums' peut être utile 


# Afficher les premières lignes de notre ensemble de données pour s'assurer que la colonne "label" a été ajoutée
data_comments.head(10)

SyntaxError: invalid syntax (<ipython-input-3-41f3a971fce1>, line 2)

$\textbf{ATTENTION: Ne regardez pas les liens dans les commentaires, ce sont des spams! ;)}$

In [ ]:
# Afficher les commentaires de spam dans les données
# N'ALLEZ PAS SUR LES LIENS !!!!! sérieusement, ce sont des spams.... 
print(data_comments["content"][data_comments["label"] == 1])

En parcourant les commentaires qui ont été étiquetés comme spam dans ces données, il semble que ces commentaires sont soit sans rapport avec la vidéo, soit comme une forme de publicité. L'expression "check out" semble être très populaire dans ces commentaires.

## Summary Statistics and Data Cleaning

Le tableau ci-dessous montre que cet ensemble de données se compose de $1959$ commentaires youtube, dont environ $49\%$ sont des commentaires légitimes et environ $51\%$ sont du spam. Cette grande variation de classes dans notre ensemble de données nous aidera à tester l'exactitude de nos algorithmes sur l'ensemble des données de test. 

La longueur moyenne de chaque commentaire est d'environ $96$ caractères, ce qui représente environ $15$ mots en moyenne par commentaire.

In [ ]:
# Ajouter une nouvelle colonne pour la longueur de chaque commentaire
# conseils: utiliser map et lambda
data_comments['length'] = 

# Permet d'afficher un tableau avec plusieurs données statistiques (mean, stdev, min, max)
data_comments[["label","length"]].describe()

Pour notre algorithme de classification Naive Bayes, nous diviserons les données en deux parties: entrainement et tests. La partie d'entrainement sera utilisé pour former l'algorithme de classification du spam, et l'ensemble de test ne sera utilisé que pour tester sa précision. 

En général, La partie d'entrainement devrait être plus grand que La partie de test et les deux devraient provenir de la même population (la population dans notre cas est Youtube commentaires pour les vidéos musicales). 

**Nous sélectionnerons au hasard $75\%$ des données pour la formation et $25\%$ des données pour les tests.**

In [ ]:
# Séparons les données en 2 groupes ! (75% training, 25% test)

# Ceci nous permet d'obtenir la même allocation aléatoire pour chaque série de codes. RTFM if you want ;)
np.random.seed(2019)

# Ajout d'un vecteur colonne 'uniform' de nombres générés aléatoirement entre 0 et 1 
# Astuce : dans numpy, il existe une méthode pour prélever un échantillon à partir d'une distribution uniforme.
data_comments["uniform"] = 

# Comme le nombre dans notre colonne 'uniform' est distribué uniformément, 
# environ 75 % de ces chiffres devraient être inférieurs à 0,75 %, prenons ces 75 %.
data_comments_train = 

# Même chose pour les 25 % de ces numéros qui sont supérieurs à 0,75
data_comments_test = 



In [ ]:
# Vérifiez que les données d'entraînement contiennent à la fois des commentaires de spam et de ham
data_comments_train["label"].describe()

In [ ]:
# Même chose pour le data test
data_comments_test["label"].describe()

Les données d'entrainement et de test ont toutes deux un bon mélange de spam et de ham, nous sommes donc prêts à passer à la formation sur le classificateur Naive Bayes. 

In [4]:
# Joindre tout les commentaires dans une seule et même liste
# Astuce: 'separator'.join(list)
training_list_words = 

# Diviser la liste des commentaires en une liste de mots uniques
# Astuce: set() and sorted()
train_unique_words = 

# Nombre de mots uniques dans le data training
vocab_size_train = 

SyntaxError: invalid syntax (<ipython-input-4-dd2d22f44c10>, line 3)

In [ ]:
# Description résumée des commentaires
print('Unique words in training data: %s' % vocab_size_train)
print('First 5 words in our unique set of words: \n % s' % list(train_unique_words)[1:6])

Cela devrait ressember à quelquechose comme ça:

```Unique words in training data: 5898
First 5 words in our unique set of words: 
['now!!!!!!', 'yellow', 'four', '/>.Pewdiepie', 'Does']```


Actuellement, "now !!" et "Now !!!!!!", ainsi que "DOES", "DoEs", et "does" sont tous considérés comme des mots uniques. Pour la classification du spam, il est probablement préférable de traiter légèrement les données pour en améliorer l'exactitude. Dans notre cas, nous pouvons nous concentrer sur les lettres et les chiffres, ainsi que convertir tous les commentaires en minuscules.

In [ ]:
# Garder uniquement les chiffres et les lettres
# Astuce: utiliser regex et les list comprehension
train_unique_words = 

# Convertir toutes les lettres en minuscule
# Astuce: set() ?
train_unique_words = 

# Nombre de mots uniques dans le data training
vocab_size_train = 

# Description résumée des commentaires
print('Unique words in processed training data: %s' % vocab_size_train)
print('First 5 words in our processed unique set of words: \n % s' % list(train_unique_words)[1:6])

## Naive Bayes for Spam Classification

Ok, alors voilà le plan :

- Tout d'abord, nous avons séparer nos données de formation en 2 sous-ensembles : training et test.

- puis nous allons créer plusieurs fonctions pour vérifier combien de fois chaque mot est apparu dans le spam et non dans les commentaires de spam, 
    - et vérifier la probabilité que chaque mot apparaisse dans le spam/non spam

- alors les 2 fonctions les plus importantes : train() et classify()

- Et enfin, nous allons vérifiez l'exactitude de nos prédictions.

Passons au code !

In [ ]:
# On initialise des dictionnaire avec des mots de commentaires comme "keys", et leur étiquette comme "value".
trainPositive = dict()
trainNegative = dict()

# On initialise ces variables à zéro
positiveTotal = 
negativeTotal = 

# Même chose, mais en float ;) 
pSpam = 
pNotSpam = 

# Laplace smoothing
alpha = 1

In [ ]:
#def initialize_dicts():

# # On initialise les dictionnaires avec 0 comme valeur 
for word in train_unique_words:
    # skip empty words('' and ' ')
    if word ... or ...:
        continue
    
    # Pour le moment, tout est classifier comme ham (légitime)
    trainPositive[word] = 0
    trainNegative[word] = 0

In [5]:
# Compter le nombre de fois que le mot dans le commentaire apparaît dans les commentaires de spam et ham
def processComment(comment,label):
    global positiveTotal
    global negativeTotal
    
    # Séparer le commentaire en liste de mots
    comment = 
    
    # Pour chaque mot du commentaire
    for ...
        
        #Checker si le mot est bien dans la base de donnée
        
        #Checker si ce n'est pas un '' ou ' '
        
        
        # Checker si le mot n'est pas du spam (ham)
        
            
            # Incrémenter le nombre de fois que le mot apparaît dans les commentaires non spam
            
            
        # spam comments
        
            
            # Incrémenter le nombre de fois que le mot apparaît dans les commentaires spam
            

SyntaxError: invalid syntax (<ipython-input-5-d350d815e653>, line 7)

##METTRE LA FORMULE



In [ ]:
# Ici, on a la fonction qui va calculer la Prob(word|spam) et Prob(word|ham)
def conditionalWord(word,label):
   
    # Paramètre de lissage de Laplace (Laplace Smoothing)
    # Rappel : pour avoir accès à une variable globale à l'intérieur d'une fonction 
    # vous devez le spécifier en utilisant le mot 'global'.
    
    
    # word in ham comment
    if(label == 0):
        # Calculer Prob(word|ham)
        
    
    # word in spam comment
    else:
        
        # Calculer Prob(word|ham)
       

In [ ]:
# Ici, on a la fonction qui va calculer la Prob(spam|comment) or Prob(ham|comment)
def conditionalComment(comment,label):
    
    # On initialise la probabilité conditionelle
    prob_label_comment = 1.0
    
    # On sépare le commentaire en liste de mots
    
    
    # Pour chaque mot du commentaire
    for ...
        
        # Calculer la valeur de P(label|comment)
        # On suppose ici qu'on a une independance conditionnelle (p(A) * p(B))
        
    
    return prob_label_comment

In [ ]:
# Calculer plusieurs probabilités conditionnelles dans les données d'entraînement
def train():
    # Rappel: on aura besoin de pSpam et pNotSpam ici ;) 


    # Initialisation de nos variables: le nombre total de commentaires et le nombre de commentaires de spam 
    total = 0.0
    num_spam = 0.0
    
    print('Starting training ...')
    # Passez en revue chaque commentaire dans les données d'entraînement 
    for ...
        
       # Vérifiez si le commentaire est du spam ou non (ham)
    
       # Incrémenter les valeurs selon que le commentaire est du spam ou non
        
       # Mettre à jour le dictionnaire du spam et ham
    
    
    # Calcule des probabilitées a priori, P(spam), P(ham)
    pSpam = 
    pNotSpam = 
    print('Training done')

In [ ]:
# Lancer notre fonction train de Naive Bayes
train()

In [ ]:
# Classifier les commentaires sont du spam ou ham
def classify(comment):
    
    # get global variables
    
    
    # Calculer la valeur proportionnelle à Pr(comment|ham)
    isNegative = 
    
    # Calculer la valeur proportionnelle à Pr(comment|spam)
    isPositive = 
    
    # Output -> True = spam, False = ham en fonction des 2 variables calculées précédemment (il faut comparer les variables)
    return 

In [ ]:
# Initialiser la prédiction du spam dans les données de test
prediction_test = []

# Obtenez la précision des prédictions sur les données d'essai
for ...

    # ajouter un commentaire classifié à la liste prediction_test 
    

# Checker la précision: 
# D'abord le nombre de prédictions correctes 
correct_labels = 
# Ensuite la moyenne des prédictions correctes
test_accuracy = 

#print prediction_test
print("Proportion of comments classified correctly on test set: %s" % test_accuracy)

Essayons d'écrire quelques commentaires pour voir s'ils sont classés comme spam ou ham. 

Rappelez-vous que le "True" est pour les commentaires de spam, et "False" est pour les commentaires ham. 
Essayez vous même !

In [ ]:
# spam
classify("Guys check out my new chanell")

In [ ]:
# spam
classify("I have solved P vs. NP, check my video https://www.youtube.com/watch?v=dQw4w9WgXcQ")

In [ ]:
# ham
classify("I liked the video")

In [ ]:
# ham
classify("Its great that this video has so many views")

### Pour aller plus loin...
## Extending Bag of Words by Using TF-IDF

Jusqu'à présent, nous avons utilisé le modèle du Bag of Words pour représenter les commentaires en tant que vecteurs. Le "Bag of Words" est une liste de tous les mots uniques trouvés dans les données training, alors chaque commentaire peut être représenté par un vecteur qui contient la fréquence de chaque mot unique qui apparaît dans le commentaire.

Par exemple, si les données training contiennent les mots $(hi, how, how, my, grade, are, you),$ alors le texte "how are you you" peut être représenté par $(0,1,0,0,0,1,2).$ La principale raison pour laquelle nous faisons cela dans notre application est que les commentaires peuvent varier en longueur, mais la longueur des mots uniques reste fixe.

Dans notre contexte, le TF-IDF est une mesure de l'importance d'un mot dans un commentaire par rapport à tous les mots de nos données de formation. Par exemple, si un mot tel que "the" apparaissait dans la plupart des commentaires, le TF-IDF serait petit car ce mot ne nous aide pas à faire la différence entre les commentaires spam et ham. Notez que "TF" signifie "Term Frequency" et "IDF" signifie "Inverse Document Frequency".

En particulier, "TF" indiqué par $tf(w,c)$ est le nombre de fois que le mot $w$ apparaît dans le commentaire donné $c$. Alors que "IDF" est une mesure de la quantité d'informations qu'un mot donné fournit pour différencier les commentaires. PLus précisement, $IDF$ est formulé comme ceci:


>$idf(w, D) = log(\frac{\text{Number of comments in train data $D$}}{\text{Number of comments containing the word $w$}}).$ 


Pour combiner "TF" et "IDF" ensemble, nous prenons simplement le produit, donc:


>$$TFIDF = tf(w,c) \times idf(w, D) = (\text{Number of times $w$ appears in comment $c$})\times log(\frac{\text{Number of comments in train data $D$}}{\text{Number of comments containing the word $w$}}).$$


Maintenant, le $TF-IDF$ peut être utilisé pour pondérer les vecteurs qui résultent de l'approche "Bag of Words".

Par exemple, supposons qu'un commentaire contienne "ceci" 2 fois, donc $tf = 2$. 
Si nous avions alors 1000 commentaires dans nos données de formation, et que le mot "ceci" apparaît dans 100 commentaires, $idf = log(1000/100) = 2.$. 

Par conséquent, dans cet exemple, le poids TF-IDF serait de $2*2 = 4$ pour le mot "ceci" apparaît deux fois dans un commentaire particulier. Pour incorporer TF-IDF dans le réglage des baies naïves, nous pouvons calculer :

>$$Pr(word|spam) = \frac{\sum_{\text{c is spam}}TFIDF(word,c,D)}{\sum_{\text{word in spam c}}\sum_{\text{c is spam}}TFIDF(word,c,D)+ \text{Number of unique words in data}},$$ 

>where $TFIDF(word,c,D) = TF(word,c) \times IDF(word,data).$ 

In [ ]:
# Calculer TFIDF(word, comment, data)
def TFIDF(comment, train):
    
    # Diviser le commentaire en une liste de mot
    comment = 
    
    # Initiailiser tf-idf selon la longueur du commentaire
    tfidf_comment = 
    
    # Initiailiser nombre de commentaires contenant un mot
    num_comment_word = 0
    
    # Initialiser l'index pour les mots dans le commentaire
    word_index = 0
    
    # Pour chaque mot du commentaire
    for...
        
        # Calculer la fréquence des termes (tf)
        # Compter la fréquence du mot dans les commentaires
        tf = 
        
        # Trouver le nombre de commentaires contenant un mot
        for ...
            
            # Incrémenter le compteur de mots si le mot trouvé dans le commentaire
            if ...
        
        # Calculer la fréquence du document inverse (idf)
        # log(Nombre total de commentaires/nombre de commentaires avec mot)
        idf = 
        
        # Mettre a jour le poids tf-idf du mot
        
        
        # Réinitialiser le nombre de commentaires contenant un mot
        
        
        # Passer au mot suivant dans le commentaire
        
        
    return tfidf_comment

In [ ]:
TFIDF("Check out my new music video plz",data_comments_train)

In [ ]:
# Et maintenant, implémente TFIDF avec ta fonction de classification
# Have fun :D

